In [49]:
#!pip install tensorflow_hub
#!pip install tensorflow_text
#!pip install numpy
#!pip install tensorflow
#!pip install pandas
#!pip install SenticNet
#!pip install tensorflow.keras 

In [50]:
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import tensorflow as tf
import pandas as pd
from senticnet.senticnet import SenticNet
import re
import string
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
#github test first commit

In [51]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [52]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [53]:
# input : 126 words as sentence  if exceeds 126, split it, if less than 126 apply padding of 0.
# output : 128 * 768

def bert_process(text_to_process):
    text_preprocessed = bert_preprocess_model(text_to_process)

    inputk = {'input_word_ids' : [], 'input_mask': [], 'input_type_ids': []}
    
    for i in text_preprocessed.keys():
        index = 0

        for arr in text_preprocessed[i]:
            arr = list(arr.numpy())
            arr = list(filter(lambda num: num != 0, arr))
            if len(inputk[i]) == 0:
                inputk[i].append(arr)
            else:
                inputk[i][index] = list(filter(lambda num: num != 0, inputk[i][index]))
                if len(inputk[i][index]) + len(arr) > 128:
                    inputk[i].append(arr)
                    lengthwithoutzero = len(inputk[i][index])
                    padd = [0]*(128-lengthwithoutzero)
                    inputk[i][index].extend(padd) 
                    index += 1
                else:
                    inputk[i][index].extend(arr)             
        lengthwithoutzero = len(inputk[i][index])
        padd = [0]*(128-lengthwithoutzero)
        inputk[i][index].extend(padd)

    inputk["input_mask"] = tf.convert_to_tensor(inputk["input_mask"])
    inputk["input_type_ids"] = tf.convert_to_tensor(inputk["input_type_ids"])
    inputk["input_word_ids"] = tf.convert_to_tensor(inputk["input_word_ids"])

    bert_results = bert_model(inputk)
    tempres = (bert_results["sequence_output"].numpy()).tolist()
    
    return tempres

In [54]:
def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    # words = words.translate(words.maketrans('', '', string.punctuation))
    return words

In [55]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|watch)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("|||","<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [56]:
MBTI = pd.read_csv('mbti_1.csv')
data = []

In [57]:
# sample is number of samples being used.
# total data = 
# 1000 sample 8m 25s
sample = 200

for n in range(1,int(sample/100) + 1):
    for post in MBTI["posts"][100*(n-1):(100 * n)]:
        sentence = clean_text(str(post))
        sentence = split_into_sentences(sentence)
        # processedinput = bert_process(sentence)
        if len(sentence) < 50:
            temp = [""] * (50-len(sentence))
            sentence = sentence + temp
        processedinput = bert_process(sentence)
        data.append(processedinput[:6])

In [58]:
print(len(data[0]))
print(len(processedinput[0]))
print(len(processedinput[0][0]))

6
128
768


In [59]:
print(len(data))
for i in data:
    if len(i) != 6:
        print(len(i))

200


In [60]:
# labels = []
IElabels = []
NSlabels = []
FTlabels = []
JPlabels = []

templabels = MBTI["type"]
# labelmap = {"INTJ" : 0.0, "INTP": 1.0, "ENTJ": 2.0, "ENTP" : 3.0, "INFJ": 4.0, "INFP": 5.0, "ENFJ": 6.0, "ENFP": 7.0, "ISTJ": 8.0, "ISFJ": 9.0, "ESTJ": 10.0, "ESFJ": 11.0, "ISTP": 12.0, "ISFP": 13.0, "ESTP": 14.0, "ESFP": 15.0}
# labeindex = ["INTJ", "INTP", "ENTJ", "ENTP", "INFJ", "INFP", "ENFJ", "ENFP", "ISTJ", "ISFJ", "ESTJ", "ESFJ", "ISTP", "ISFP", "ESTP", "ESFP"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}

for l in templabels:
    IElabels.append(labelmap[l[0]])
    NSlabels.append(labelmap[l[1]])
    FTlabels.append(labelmap[l[2]])
    JPlabels.append(labelmap[l[3]])


#split train, val, test data


In [61]:
Y_IE_label = IElabels[:sample]
Y_NS_label = NSlabels[:sample]
Y_FT_label = FTlabels[:sample]
Y_JP_label = JPlabels[:sample]

In [62]:
# sum of val data and test data
TEST_SIZE = 0.3
# portion of val from test
#VAL_SIZE = 0.9

In [63]:
# split IE_label to test train
X_train, X_IEtest, Y_train, Y_IEtest = train_test_split(
    data, Y_IE_label, test_size=TEST_SIZE, random_state=42)

In [64]:
# split IE_label to test val
# X_IE_val, X_IE_test, Y_IE_val, Y_IE_test = train_test_split(
#    X_IEtest, Y_IEtest, test_size = VAL_SIZE, random_state=42)

In [65]:
# split NS_label to test train
#X_train, X_NStest, Y_train, Y_NStest = train_test_split(
#    data, Y_NS_label, test_size = TEST_SIZE, random_state=42)

In [66]:
# split NS_label to test val
#X_NS_val, X_NS_test, Y_NS_val, Y_NS_test = train_test_split(
#    X_NStest, Y_NStest, test_size = VAL_SIZE, random_state=42)

In [67]:
# split FT_label to test train
#X_train, X_FTtest, Y_train, Y_FTtest = train_test_split(
#    data, Y_FT_label, test_size = TEST_SIZE, random_state=42)

In [68]:
# split FT_label to test val
#X_FT_val, X_FT_test, Y_FT_val, Y_FT_test = train_test_split(
#    X_FTtest, Y_FTtest, test_size = VAL_SIZE, random_state=42)

In [69]:
# split JP_label to test train
#X_train, X_JPtest, Y_train, Y_JPtest = train_test_split(
#    data, Y_JP_label, test_size = TEST_SIZE, random_state=42)

In [70]:
# split JP_label to test val
#X_JP_val, X_JP_test, Y_JP_val, Y_JP_test = train_test_split(
#    X_JPtest, Y_JPtest, test_size = VAL_SIZE, random_state=42)

# IE model

In [71]:
#convert to tensor
X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)
#X_IE_val = tf.convert_to_tensor(X_IE_val)
#Y_IE_val = tf.convert_to_tensor(Y_IE_val)
X_IE_test = tf.convert_to_tensor(X_IEtest)
Y_IE_test = tf.convert_to_tensor(Y_IEtest)

In [72]:
# model constructor for CNN classification
def model_fn() :
  model = models.Sequential()
  model.add(layers.Conv2D(32, (1, 3), activation='relu', input_shape=X_train.shape[1:]))
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.Conv2D(32, (3, 1), activation='relu'))
  
  # dropout
  # model.add(layers.Dropout(0.3))
  # fully connected layer
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='sigmoid'))
  model.add(layers.Dense(1, activation='sigmoid'))
  #
  return model

In [84]:
IEmodel = model_fn()
IEmodel.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 6, 126, 32)        73760     
                                                                 
 conv2d_19 (Conv2D)          (None, 4, 124, 32)        9248      
                                                                 
 conv2d_20 (Conv2D)          (None, 2, 124, 32)        3104      
                                                                 
 flatten_6 (Flatten)         (None, 7936)              0         
                                                                 
 dense_12 (Dense)            (None, 128)               1015936   
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,102,177
Trainable params: 1,102,177
No

In [85]:
IEmodel.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-IE-model.h5', 
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                 restore_best_weights=True)

history = IEmodel.fit(X_train, Y_train, epochs=20, batch_size = 32,
                    validation_data=(X_IE_test, Y_IE_test))


Epoch 1/20
5/5 [==============================] - 2s 229ms/step - loss: 0.6414 - accuracy: 0.6429 - val_loss: 0.4848 - val_accuracy: 0.8167
Epoch 2/20
5/5 [==============================] - 1s 186ms/step - loss: 0.5566 - accuracy: 0.7571 - val_loss: 0.5517 - val_accuracy: 0.8167
Epoch 3/20
5/5 [==============================] - 1s 186ms/step - loss: 0.4713 - accuracy: 0.7643 - val_loss: 0.4607 - val_accuracy: 0.8167
Epoch 4/20
5/5 [==============================] - 1s 188ms/step - loss: 0.3742 - accuracy: 0.7714 - val_loss: 0.4566 - val_accuracy: 0.8167
Epoch 5/20
5/5 [==============================] - 1s 180ms/step - loss: 0.2752 - accuracy: 0.8929 - val_loss: 0.4372 - val_accuracy: 0.8167
Epoch 6/20
5/5 [==============================] - 1s 185ms/step - loss: 0.1491 - accuracy: 0.9643 - val_loss: 0.4697 - val_accuracy: 0.7833
Epoch 7/20
5/5 [==============================] - 1s 181ms/step - loss: 0.0638 - accuracy: 1.0000 - val_loss: 0.4538 - val_accuracy: 0.8167
Epoch 8/20
5/5 [====

In [86]:
IEmodel.evaluate(X_IE_test, Y_IE_test)

2/2 [==============================] - 0s 61ms/step - loss: 0.7995 - accuracy: 0.8167


[0.7995364665985107, 0.8166666626930237]